In [1]:
import numpy as np
import pandas as pd

In [2]:
import pulp

In [3]:
from src.modeling import *

In [4]:
def of_net_income(_cost, _vars, _crops_year):
    return pulp.lpSum([getattr(_cost, _crops_year[i]) * _vars[i] for i in _crops_year])

def of_ua_total(_vars, _crops_year):
    return pulp.lpSum([_vars[i] for i in _crops_year])

def of_crop_diversity(_vars, _crops_year):
    return pulp.lpSum([_vars[i] for i in _crops_year])

In [5]:
variables = pd.read_csv('dados/al_rasheed/variables.csv', sep=';')
income = pd.read_csv('dados/al_rasheed/net_income.csv', sep=';')
constraints = pd.read_csv('dados/al_rasheed/constraints.csv', sep=';')
rotations = pd.read_csv('dados/al_rasheed/rotations.csv', sep=';').fillna(0)

In [6]:
YEARS = 2

In [7]:
crops = variables.variable.to_list()

crops_year_integer = dict(zip([f"y{i}_int_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))

crops_year_binary = dict(zip([f"y{i}_bin_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))

In [8]:
variables.set_index(keys='variable', inplace=True)

integer_vars = create_integer_vars(crops_year_integer, variables)

binary_vars = create_binary_vars(crops_year_binary, variables)

In [9]:
ofs = [of_net_income(income.loc[0,:], integer_vars, crops_year_integer),
       of_ua_total(integer_vars, crops_year_integer),
       of_crop_diversity(binary_vars, crops_year_binary)]

In [10]:
min_max_ofs = []
for of in ofs:
    vl = []
    for opt in [pulp.LpMinimize, pulp.LpMaximize]:
        model = pulp.LpProblem("Modelo_1", opt)
        model += of
        const_idx = add_constraints(model, integer_vars, constraints, crops_year_integer, len(crops), YEARS, const_id = 0)
        const_idx = add_binary_correlation(model, binary_vars, integer_vars, 113, const_id = const_idx)
        const_idx = add_crop_rotation_constraints(model, integer_vars, crops_year_integer, rotations, YEARS, crops, 'y{}_int_{}', const_idx)

        solution = model.solve(pulp.GLPK())

        vl.append(pulp.value(model.objective))
    
    min_max_ofs.append(vl)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/8080702f67d04ae298ed0dc7ca76ca86-pulp.lp -o /tmp/8080702f67d04ae298ed0dc7ca76ca86-pulp.sol
Reading problem data from '/tmp/8080702f67d04ae298ed0dc7ca76ca86-pulp.lp'...
88 rows, 48 columns, 430 non-zeros
48 integer variables, 24 of which are binary
294 lines were read
GLPK Integer Optimizer 5.0
88 rows, 48 columns, 430 non-zeros
48 integer variables, 24 of which are binary
Preprocessing...
24 constraint coefficient(s) were reduced
64 rows, 48 columns, 242 non-zeros
48 integer variables, 24 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.260e+03  ratio =  2.260e+03
GM: min|aij| =  4.453e-01  max|aij| =  2.246e+00  ratio =  5.042e+00
EQ: min|aij| =  2.100e-01  max|aij| =  1.000e+00  ratio =  4.761e+00
2N: min|aij| =  1.104e-01  max|aij| =  1.562e+00  ratio =  1.416e+01
Constructing initial basis...
Size of triangular part is 64
Solving LP relaxation...
GLPK Simplex Optimizer 5.

In [11]:
ofs_norn = [ min_max_normalization(ofs[idx], min_max_ofs[idx]) for idx in range(len(ofs))]

In [12]:
model = pulp.LpProblem("Modelo_1", pulp.LpMaximize)

In [13]:
model += ofs_norn[0]

In [14]:
const_idx = add_constraints(model, integer_vars, constraints, crops_year_integer, len(crops), YEARS, const_id = 0)
const_idx = add_binary_correlation(model, binary_vars, integer_vars, 113, const_id = const_idx)
const_idx = add_crop_rotation_constraints(model, integer_vars, crops_year_integer, rotations, YEARS, crops, 'y{}_int_{}', const_idx)

In [15]:
solution = model.solve(pulp.GLPK())

print("\nStatus = {} \nValue = {}".format(pulp.LpStatus[solution], pulp.value(model.objective)))

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/eb4366b7f66f462faf67e4db028fd1f0-pulp.lp -o /tmp/eb4366b7f66f462faf67e4db028fd1f0-pulp.sol
Reading problem data from '/tmp/eb4366b7f66f462faf67e4db028fd1f0-pulp.lp'...
88 rows, 48 columns, 430 non-zeros
48 integer variables, 24 of which are binary
298 lines were read
GLPK Integer Optimizer 5.0
88 rows, 48 columns, 430 non-zeros
48 integer variables, 24 of which are binary
Preprocessing...
24 constraint coefficient(s) were reduced
64 rows, 48 columns, 242 non-zeros
48 integer variables, 24 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.260e+03  ratio =  2.260e+03
GM: min|aij| =  4.453e-01  max|aij| =  2.246e+00  ratio =  5.042e+00
EQ: min|aij| =  2.100e-01  max|aij| =  1.000e+00  ratio =  4.761e+00
2N: min|aij| =  1.104e-01  max|aij| =  1.562e+00  ratio =  1.416e+01
Constructing initial basis...
Size of triangular part is 64
Solving LP relaxation...
GLPK Simplex Optimizer 5.

12559922.4 | 6332504

In [16]:
model

Modelo_1:
MAXIMIZE
8.774835954441831e-05*y1_int_Autumn_Potatoes + 8.235346040205779e-05*y1_int_Beetroot + 0.00017926664861000425*y1_int_Broad_Bean + 7.370862201731139e-05*y1_int_Cloves + 7.390361837185454e-05*y1_int_Cucumbers + 3.3474374196574393e-05*y1_int_Green_Onion + 0.00011374787348350522*y1_int_Maize + 0.00020214622087640072*y1_int_Okra + 8.521340693535733e-05*y1_int_Sunflower + 0.00010172309828667753*y1_int_Tomatoes + 8.872334131713407e-05*y1_int_Watermelon + 6.824872409010313e-06*y1_int_Wheat + 8.774835954441831e-05*y2_int_Autumn_Potatoes + 8.235346040205779e-05*y2_int_Beetroot + 0.00017926664861000425*y2_int_Broad_Bean + 7.370862201731139e-05*y2_int_Cloves + 7.390361837185454e-05*y2_int_Cucumbers + 3.3474374196574393e-05*y2_int_Green_Onion + 0.00011374787348350522*y2_int_Maize + 0.00020214622087640072*y2_int_Okra + 8.521340693535733e-05*y2_int_Sunflower + 0.00010172309828667753*y2_int_Tomatoes + 8.872334131713407e-05*y2_int_Watermelon + 6.824872409010313e-06*y2_int_Wheat + -0.